In [ ]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import folium
import folium.plugins
import logging
from eo_tools.util import explore_products, show_s2_color
from eo_tools.S2 import process_s2_tiles
from zipfile import ZipFile

import matplotlib.pyplot as plt

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

# change your data directories
path_data = "/data"
path_s5p = f"{path_data}/S5P"
path_processed = f"{path_data}/res"

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

# Search

In [ ]:
# load a geometry
# file_aoi = "../data/Bretagne_AOI.geojson"
file_aoi = "../data/Nevado-del-Ruiz.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S5P_L2_SO2",
    "start": "2024-02-04",
    "end": "2024-02-06",
    "geom": shp
}
results = dag.search_all(**search_criteria)

Getting unique dates to help with product exploration

In [ ]:
def get_unique_dates(results):
    import numpy as np

    dates = []
    for p in results:
        dates.append(p.properties["startTimeFromAscendingNode"])
    return np.unique(dates).tolist()

dates = get_unique_dates(results)
print(f"unique dates:")
print(", \n".join(dates))

# Visualize product footprints on a map

In [ ]:
# show all results
# explore_products(results, shp)

# filter by date
results_filtered = results.filter_date("2024-02-04T18:00", "2024-02-04T19:00") 
explore_products(results_filtered, shp)

# Download

In [ ]:
# re-run to get only links
dl = dag.download_all(results_filtered, outputs_prefix=path_s5p, extract=True, delete_archive=True)

# Regridding and saving COG

In [ ]:
from eo_tools.S5P import s5p_so2_to_cog
file_re = s5p_so2_to_cog(dl[0], path_processed, res_km=5)

In [ ]:
from eo_tools.util import show_cog
show_cog(file_re, rescale="0,0.01", colormap_name="reds")